<center><img src="https://synapsedatascience.com.br/wp-content/uploads/2020/12/logo-jupyter.png" width="300"/></center>

# 1. Motrar os dados no mapa

Vamos estudar o passo a passo para mostrar os dados nos mapas.

Para criar os mapas e construir a nossa visualização, vamos utilizar a biblioteca **folium**.

Você pode instalar a biblioteca folium através do conda ou do pip:

In [3]:
# instalando a biblioteca folium com conda
!conda install folium -c conda-forge -y

# instalando a biblioteca folium com pip
#!pip install folium

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [4]:
# mostrar a versão da biblioteca
!conda list folium

# packages in environment at /home/carlos/miniconda3/envs/synapse:
#
# Name                    Version                   Build  Channel
folium                    0.12.0             pyhd8ed1ab_0    conda-forge


Vamos importa as bibliotecas **folium** e **geopandas**.

In [3]:
# importar a biblioteca geopandas
import geopandas as gpd

# importar a biblioteca folium
import folium

Vamos primeiro criar um mapa simples.

In [4]:
# criar um mapa simples usando folium
fmap = folium.Map()

fmap

Com apenas uma função já conseguimos criar o nosso mapa. Nele nós conseguimos realizar as operações de **zoom** e **movimento com mouse**.

E para mostrar os nosso dados?

Vamos iniciar carregando os nosso dados da capital de São Paulo.

In [5]:
# carregando os dados de geometria
gdf_roubos_sp = gpd.read_file("dados/roubo_celular_capital.json", driver="GeoJSON")

# carregando os dados de roubo
gdf_geometria_sp = gpd.read_file("dados/capital_são_paulo.json", driver="GeoJSON")

Vamos criar um novo mapa e mostrar os dados.

In [6]:
# separar parte dos dados de roubo
gdf_roubos_slice = gdf_roubos_sp.iloc[:100]

# criar o mapa
fmap = folium.Map()

# criar uma feature geojson com os dados
feat_geojson = folium.features.GeoJson(gdf_roubos_slice)

# adicionar os dados no mapa
fmap.add_child(feat_geojson)

# mostrar o mapa
fmap

Os nossos dados foram mostrados corretamente, mas foi preciso fazer um zoom no mapa para observar melhor. Como podemos deixar o mapa mais responsivo no zoom?

In [7]:
# vamos calcular o valor médio de latitude e longitude dos nosso dados
media_latitude = gdf_roubos_slice['LATITUDE'].mean()
media_longitude = gdf_roubos_slice['LONGITUDE'].mean()

# criar o mapa
fmap = folium.Map(location=[media_latitude, media_longitude])

# criar uma feature geojson com os dados
feat_geojson = folium.features.GeoJson(gdf_roubos_slice)

# adicionar os dados no mapa
fmap.add_child(feat_geojson)

# mostrar o mapa
fmap

# 2. Criar mapas de agrupamentos (_clusters_)

E para mostrar todos os nosso dados? Adicionando uma grande quantidade de dados no nosso mapa pode dificultar a visualização, por isso vamos utilizar um plugin do folium para melhorar a visualização fazendo agrupamentos dinâmicos.

In [8]:
# importar o plugin FastMarkerCluster
from folium.plugins import FastMarkerCluster

In [9]:
# criar o mapa utilizando o parâmetro location com as médias
fmap = folium.Map(location=[media_latitude, media_longitude])

# criar o marcador de clusterização
mc = FastMarkerCluster(gdf_roubos_sp[['LATITUDE', 'LONGITUDE']])

# adicionar marcador no mapa
fmap.add_child(mc)

# mostrar o mapa
fmap

Como podemos melhorar o nosso mapa?

1. Trocar o mapa base;
2. Adicionar as fronteiras da nossa cidade;
3. Trocar o estilo dos dados.

In [10]:
# mostrar o geodataframe de são paulo
gdf_geometria_sp

,CD_MUN,NM_MUN,SIGLA_UF,AREA_KM2,geometry
0,3550308,São Paulo,SP,1521.11,"POLYGON ((-46.63386 -23.42914, -46.63341 -23.4..."


In [11]:
# criar o mapa, trocando o tiles para 'cartodbpositron'
fmap = folium.Map(location=[media_latitude, media_longitude], tiles='cartodbpositron')

# criar o GeoJson da nossa geometria para adicionar ao mapa
limites = folium.features.GeoJson(gdf_geometria_sp, 
                                  style_function=lambda feature: {
                                      'color': 'black',
                                      'weight' : 2,
                                      'fillOpacity' : 0.0
                                  })
# adicionar limites no nosso mapa
fmap.add_child(limites)

# criar o marcador de clusterização
mc = FastMarkerCluster(gdf_roubos_sp[['LATITUDE', 'LONGITUDE']])

# adicionar marcador no mapa
fmap.add_child(mc)

# mostrar o mapa
fmap

Podemos salvar os mapas criados com a biblioteca folium facilmente usando o método __save__.

In [44]:
# salvar o nosso mapa em um arquivo html
fmap.save('mapas/mapa_roubos_sp.html')

Vamos abrir o nosso mapa salvo em html e ver o resultado!

# 3. Criar mapas de calor (_HeatMap_)

Para criar o nosso mapa de calor, vamos utilizar um plugin **HeatMap** da biblioteca folium.

In [12]:
# importar o plugin HeatMap
from folium.plugins import HeatMap

In [13]:
# criar o mapa com o tiles 'cartodbpositron'
fmap = folium.Map(location=[media_latitude, media_longitude], tiles='cartodbpositron')

# criar o heatmap
heat_map = HeatMap(gdf_roubos_sp[['LATITUDE', 'LONGITUDE']])

# adicionar no mapa
fmap.add_child(heat_map)

# mostrar o mapa
fmap

Vamos criar um heatmap para a região de grande são paulo.

In [50]:
# salvar o nosso mapa em um arquivo html
fmap.save('mapas/heatmap_roubos_sp.html')

# 4. Adicionar informações estatísticas no mapa

Vamos contar a quantidade de roubos de celular em cada município de grande são paulo e plotar um mapa com um popup da quantidade.

In [14]:
# carregar os municípios de grande são paulo
gdf_geometrias_gsp = gpd.read_file("dados/municipios_grande_sp.json", driver="GeoJSON")

# carregar os dados de roubo de grande são paulo
gdf_roubos_gsp = gpd.read_file("dados/roubo_celular_grande_sp.json", driver="GeoJSON")

Vamos analisar os nossos dados.

In [15]:
# mostrar os dados
gdf_geometrias_gsp.head()

,CD_MUN,NM_MUN,SIGLA_UF,AREA_KM2,geometry
0,3503901,Arujá,SP,96.167,"POLYGON ((-46.36995 -23.37809, -46.37010 -23.3..."
1,3505708,Barueri,SP,65.701,"POLYGON ((-46.83848 -23.51062, -46.83921 -23.5..."
2,3506607,Biritiba Mirim,SP,317.406,"POLYGON ((-46.09244 -23.56664, -46.09256 -23.5..."
3,3509007,Caieiras,SP,97.642,"POLYGON ((-46.82345 -23.35390, -46.82345 -23.3..."
4,3509205,Cajamar,SP,131.386,"POLYGON ((-46.90468 -23.35731, -46.90538 -23.3..."


In [16]:
# contar a quantidade de dados em cada município e salvar em uma coluna 
for index, municipio in gdf_geometrias_gsp.iterrows():
    
    # contar a quantidade de dados no município
    qtd_roubos = len(gdf_roubos_gsp[gdf_roubos_gsp.intersects(municipio.geometry)])
    
    # salvar no geodataframe
    gdf_geometrias_gsp.loc[index, 'qtd_roubos'] = qtd_roubos

In [17]:
gdf_geometrias_gsp.head(5)

,CD_MUN,NM_MUN,SIGLA_UF,AREA_KM2,geometry,qtd_roubos
0,3503901,Arujá,SP,96.167,"POLYGON ((-46.36995 -23.37809, -46.37010 -23.3...",18.0
1,3505708,Barueri,SP,65.701,"POLYGON ((-46.83848 -23.51062, -46.83921 -23.5...",51.0
2,3506607,Biritiba Mirim,SP,317.406,"POLYGON ((-46.09244 -23.56664, -46.09256 -23.5...",0.0
3,3509007,Caieiras,SP,97.642,"POLYGON ((-46.82345 -23.35390, -46.82345 -23.3...",13.0
4,3509205,Cajamar,SP,131.386,"POLYGON ((-46.90468 -23.35731, -46.90538 -23.3...",17.0


In [18]:
# vamos calcular o valor médio de latitude e longitude dos nosso dados
media_latitude_gsp = gdf_roubos_gsp['LATITUDE'].mean()
media_longitude_gsp = gdf_roubos_gsp['LONGITUDE'].mean()

# criar o mapa com o tiles 'cartodbpositron'
fmap = folium.Map(location=[media_latitude_gsp, media_longitude_gsp], tiles='cartodbpositron')

# fazer um for nos municípios e criar o geojson da geometria e o popup 
for _, municipio in gdf_geometrias_gsp.iterrows():
    
    # criar o GeoJson da nossa geometria para adicionar ao mapa
    municipio_geojson = folium.features.GeoJson(municipio.geometry,
                            style_function=lambda feature: {
                                'color': 'blue',
                                'weight' : 2,
                                'fillOpacity' : 0.1
                            })
    
    # criar o popup com as informações
    popup=folium.Popup("""
                       Município: {} <br>
                       Roubos registrados em Novembro: {} 
                       """.format(municipio.NM_MUN, str(int(municipio.qtd_roubos))), max_width=300)
    
    # adicionar o popup na geoemtria do município
    popup.add_to(municipio_geojson)
    
    # adicionar a geometria no mapa
    municipio_geojson.add_to(fmap)

# salvar o mapa
fmap.save('mapas/qtd_roubos_grande_sp.html')

Vamos abrir o nosso mapa salvo em html e ver o resultado!

### `Exercício:` construa um HeatMap com os dados de grande São Paulo.
1. Carregue os dados de eventos e geometria de grande São Paulo.
2. Inicialize o mapa usando o parâmetro location com os valores médios de latitude e longitude.
3. Adicione a fronteira da geometria no mapa.
4. Crie o HeatMap e adicione no mapa.
5. Salve o mapa em um arquivo html.

In [87]:
# vamos calcular o valor médio de latitude e longitude dos nosso dados
media_latitude = gdf_roubos_grande_sp['LATITUDE'].mean()
media_longitude = gdf_roubos_grande_sp['LONGITUDE'].mean()

# criar o mapa com o tiles 'cartodbpositron'
fmap = folium.Map(location=[media_latitude, media_longitude], tiles='cartodbpositron')

# criar o GeoJson da nossa geometria para adicionar ao mapa
limites = folium.features.GeoJson(gdf_grande_sp, 
                                  style_function=lambda feature: {
                                      'color': 'black',
                                      'weight' : 2,
                                      'fillOpacity' : 0.0
                                  })

# adicionar limites no nosso mapa
fmap.add_child(limites)

# criar o heatmap
heat_map = HeatMap(gdf_roubos_grande_sp[['LATITUDE', 'LONGITUDE']])

# adicionar no mapa
fmap.add_child(heat_map)

# salvar o mapa
fmap.save('mapas/heatmap_roubos_grande_sp.html')

## Resumo

Nesta aula nós vimos como criar mapas interativos e mostrar os dados georreferenciados. Mais especificamente 
voce deve ter aprendido:

**1)** Criar mapas com a biblioteca Folium;

**2)** Motrar dados de diferentes geometrias no mapa;

**3)** Criar mapas com cluster de dados;

**4)** Criar mapas de calor (heatmaps);

**5)** Adicionar informações estatísticas no mapa.